In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta,date
from pymongo import MongoClient

In [2]:
mongo_client=MongoClient()

dbeq=mongo_client['NSEEQ']
dbfutidx=mongo_client['NSEFUTIDX']
dbfutstk=mongo_client['NSEFUTSTK']
dboptidx=mongo_client['NSEOPTIDX']
dboptstk=mongo_client['NSEOPTSTK']

In [3]:

# ###Covert to UTC to get the correct Values
# df['Datetime']=pd.to_datetime(df['Datetime']).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
# #df['Datetime']=pd.to_datetime(df['Datetime'])
# df=df.set_index('Datetime')
# return df

In [4]:
def getdata(stock,db):
    stock=str(stock)
    mongo_client=MongoClient()
    db=mongo_client[db]
    data=db[stock].find()
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    ###Covert to UTC to get the correct Values
    df['Datetime']=pd.to_datetime(df['Datetime']).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
    #df['Datetime']=pd.to_datetime(df['Datetime'])
    df=df.set_index('Datetime')
    return df


In [5]:
testdf=getdata('ACC','NSEEQ')
startdate=str(testdf.tail(1).index[0]).split(' ')[0]   #+timedelta(days=1)).split(' ')[0]
# startdate='2021-12-01'
startdate


'2021-12-01'

In [6]:
# db=mongo_client['STAGINGDB122021']
# # data=db['SECMASTER'].find({'DATE1':{'$gt':startdate}})
# data=db['SECMASTER'].find({'SYMBOL':'ACC'})
# df=pd.DataFrame(list(data))
# df.drop("_id",axis=1,inplace=True)
# df['DATE1']=pd.to_datetime(df['DATE1'])
# dbdf=df.loc[df['DATE1']>startdate]
# # newdf=df.loc[df['SERIES']=='EQ']
# # dbdf=newdf.loc[newdf['SYMBOL']=='ACC']
# dbdf

In [7]:
#This will create series of symbols 


In [8]:
def db_insert_eq(stagingdb):
    db=mongo_client[stagingdb]
    data=db['SECMASTER'].find({'DATE1':{'$gt':startdate}})
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    newdf=df.loc[df['SERIES']=='EQ']
    symbols=newdf['SYMBOL'].unique()
    for symbol in symbols:
        data=db['SECMASTER'].find({'SERIES':'EQ','SYMBOL':symbol})
        df=pd.DataFrame(list(data))
        df.drop("_id",axis=1,inplace=True)
        df['DATE1']=pd.to_datetime(df['DATE1'])
        dbdf=df.loc[df['DATE1']>startdate]
        if not dbdf.empty:
            dbdf=dbdf.replace(to_replace='-',value=0)
            dbdf.dropna(inplace=True)
            dbdf=dbdf.rename(columns={'DATE1':'Datetime','PREV_CLOSE':'Pclose','OPEN_PRICE':'Open','HIGH_PRICE':'High'
                              ,'LOW_PRICE':'Low','CLOSE_PRICE':'Close','AVG_PRICE':'Vwap',
                              'TTL_TRD_QNTY':'Volume','DELIV_QTY':'Delivery','DELIV_PER':'PerDelivery'
                             ,'NO_OF_TRADES':'#Trades'})
            dbdf.drop(['SYMBOL','SERIES','LAST_PRICE','TURNOVER_LACS'],axis=1,inplace=True)
            dbdf['Datetime']=pd.to_datetime(dbdf['Datetime'])
            # dbdf.set_index('Datetime')
            # dbdf=dbdf.apply(pd.to_numeric)
            # dbdf.reset_index()
            data=dbdf.to_dict(orient='records')
            print(f'Inserting EQ stock data for {symbol} started')
            dbeq[symbol].insert_many(data)
            print(f'Inserting EQ stock data for {symbol} Ended')
        

In [9]:
#db_insert_eq('STAGINGDB112021')

In [10]:
def db_insert_futidx(stagingdb):
    db=mongo_client[stagingdb]
    fnodata=db['FNOMASTER'].find({'INSTRUMENT':'FUTIDX','TIMESTAMP':{'$gt':startdate}})
    fnodf=pd.DataFrame(list(fnodata))
    fnodf.drop("_id",axis=1,inplace=True)
    fnosymbols=fnodf['SYMBOL'].unique()
    for fnosymbol in fnosymbols:
        fnodata=db['FNOMASTER'].find({'INSTRUMENT':'FUTIDX','SYMBOL':fnosymbol})
        fnodf=pd.DataFrame(list(fnodata))
        fnodf.drop("_id",axis=1,inplace=True)
        fnodf['TIMESTAMP']=pd.to_datetime(fnodf['TIMESTAMP'])
        dbdf=fnodf.loc[fnodf['TIMESTAMP']>startdate]
        # dbdf=fnodf.loc[fnodf['SYMBOL']==fnosymbol]
        if not dbdf.empty:
            dbdf=dbdf.replace(to_replace='-',value=0)
            dbdf=dbdf.replace(to_replace=np.nan,value=0)
            dbdf=dbdf[['EXPIRY_DT', 'OPEN',
           'HIGH', 'LOW', 'SETTLE_PR', 'CONTRACTS',
           'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
            dbdf=dbdf.rename(columns={'SETTLE_PR':'CLOSE','TIMESTAMP':'Datetime'})
            dbdf['EXPIRY_DT']=pd.to_datetime(dbdf['EXPIRY_DT'])
            dbdf['Datetime']=pd.to_datetime(dbdf['Datetime'])
            data=dbdf.to_dict(orient='records')
            print(f'Inserting future index data for {fnosymbol} started')
            dbfutidx[fnosymbol].insert_many(data)
            print(f'Inserting future index data for {fnosymbol} ended')
        
        
    

In [11]:
#db_insert_futidx('STAGINGDB112021')

In [12]:
def db_insert_futstk(stagingdb):
    db=mongo_client[stagingdb]
    fnodata=db['FNOMASTER'].find({'INSTRUMENT':'FUTSTK','TIMESTAMP':{'$gt':startdate}})
    fnodf=pd.DataFrame(list(fnodata))
    fnodf.drop("_id",axis=1,inplace=True)
    fnosymbols=fnodf['SYMBOL'].unique()
    #Converting fno all symbols to dataframe to store in db table FUTSYMBOLS
    # symbolsdf=pd.DataFrame(fnosymbols,columns=['Symbols'])
    # symboldata=symbolsdf.to_dict(orient='records')
    # dbfutstk['FUTSYMBOLS'].insert_many(symboldata)
    
    for fnosymbol in fnosymbols:
        fnodata=db['FNOMASTER'].find({'INSTRUMENT':'FUTSTK','SYMBOL':fnosymbol})
        fnodf=pd.DataFrame(list(fnodata))
        fnodf.drop("_id",axis=1,inplace=True)
        fnodf['TIMESTAMP']=pd.to_datetime(fnodf['TIMESTAMP'])
        dbdf=fnodf.loc[fnodf['TIMESTAMP']>startdate]
        
        # dbdf=fnodf.loc[fnodf['SYMBOL']==fnosymbol]
        if not dbdf.empty:
            dbdf=dbdf.replace(to_replace='-',value=0)
            dbdf=dbdf.replace(to_replace=np.nan,value=0)
            dbdf=dbdf[['EXPIRY_DT', 'OPEN','HIGH', 'LOW', 'SETTLE_PR', 'CONTRACTS',
                       'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
            dbdf=dbdf.rename(columns={'SETTLE_PR':'CLOSE','TIMESTAMP':'Datetime'})
            dbdf['EXPIRY_DT']=pd.to_datetime(dbdf['EXPIRY_DT'])
            dbdf['Datetime']=pd.to_datetime(dbdf['Datetime'])
            data=dbdf.to_dict(orient='records')
            print(f'Inserting future stock data for {fnosymbol} started')
            dbfutstk[fnosymbol].insert_many(data)
            print(f'Inserting future stock data for {fnosymbol} Ended')
        

In [13]:
#db_insert_futstk('STAGINGDB112021')

In [14]:
def db_insert_optidx(stagingdb):
    db=mongo_client[stagingdb]
    fnodata=db['FNOMASTER'].find({'INSTRUMENT':'OPTIDX','OPEN_INT': {'$gt':0}
                                  ,'TIMESTAMP':{'$gt':startdate}})
    fnodf=pd.DataFrame(list(fnodata))
    # fnodf.drop("_id",axis=1,inplace=True)
    fnosymbols=fnodf['SYMBOL'].unique()
    for fnosymbol in fnosymbols:
        fnodata=db['FNOMASTER'].find({'INSTRUMENT':'OPTIDX','SYMBOL':fnosymbol,
                                     'OPEN_INT': {'$gt':0}})
        fnodf=pd.DataFrame(list(fnodata))
        
        
        # dbdf=fnodf.loc[fnodf['SYMBOL']==fnosymbol]
        if not dbdf.empty:
            fnodf.drop("_id",axis=1,inplace=True)
            fnodf['TIMESTAMP']=pd.to_datetime(fnodf['TIMESTAMP'])
            dbdf=fnodf.loc[fnodf['TIMESTAMP']>startdate]
            dbdf=dbdf.replace(to_replace='-',value=0)
            dbdf=dbdf.replace(to_replace=np.nan,value=0)
            dbdf=dbdf[['EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN',
                       'HIGH', 'LOW', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH',
                       'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
            dbdf=dbdf.rename(columns={'TIMESTAMP':'Datetime'})
            dbdf['EXPIRY_DT']=pd.to_datetime(dbdf['EXPIRY_DT'])
            dbdf['Datetime']=pd.to_datetime(dbdf['Datetime'])
            data=dbdf.to_dict(orient='records')
            print(f'Inserting option index data for {fnosymbol} started')
            dboptidx[fnosymbol].insert_many(data)
            print(f'Inserting option index data for {fnosymbol} Ended')
    

In [15]:
#db_insert_optidx('STAGINGDB112021')

In [16]:
# def db_insert_optstk():
#     fnodata=db['FNOMASTER'].find({'INSTRUMENT':'OPTSTK','OPEN_INT': {'$gt':0}})
#     fnodf=pd.DataFrame(list(fnodata))
#     fnodf.drop("_id",axis=1,inplace=True)
#     fnosymbols=fnodf['SYMBOL'].unique()
#     for fnosymbol in fnosymbols:
#         dbdf=fnodf.loc[fnodf['SYMBOL']==fnosymbol]
#         if not dbdf.empty:
#             dbdf=dbdf.replace(to_replace='-',value=0)
#             dbdf=dbdf.replace(to_replace=np.nan,value=0)
#             dbdf=dbdf[['EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN',
#                        'HIGH', 'LOW', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH',
#                        'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
#             dbdf=dbdf.rename(columns={'TIMESTAMP':'Datetime'})
#             dbdf['EXPIRY_DT']=pd.to_datetime(dbdf['EXPIRY_DT'])
#             dbdf['Datetime']=pd.to_datetime(dbdf['Datetime'])
#             data=dbdf.to_dict(orient='records')
#             dboptstk[fnosymbol].insert_many(data)

In [17]:
def db_insert_optstk(stagingdb):
    db=mongo_client[stagingdb]
    data=dbfutstk['FUTSYMBOLS'].find()
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    
    symbols=list(df['Symbols'].unique())
    for symbol in symbols:
        fnodata=db['FNOMASTER'].find({'SYMBOL':symbol,
                                      'INSTRUMENT':'OPTSTK','OPEN_INT': {'$gt':0}})
        dbdf=pd.DataFrame(list(fnodata))
        
        
        if not dbdf.empty:
            dbdf['TIMESTAMP']=pd.to_datetime(dbdf['TIMESTAMP'])
            dbdf=dbdf.loc[dbdf['TIMESTAMP']>startdate]
            dbdf.drop("_id",axis=1,inplace=True)
            dbdf=dbdf.replace(to_replace='-',value=0)
            dbdf=dbdf.replace(to_replace=np.nan,value=0)
            dbdf=dbdf[['EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN',
                       'HIGH', 'LOW', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH',
                       'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
            dbdf=dbdf.rename(columns={'TIMESTAMP':'Datetime'})
            dbdf['EXPIRY_DT']=pd.to_datetime(dbdf['EXPIRY_DT'])
            dbdf['Datetime']=pd.to_datetime(dbdf['Datetime'])
            data=dbdf.to_dict(orient='records')
            print(f'Inserting option stock data for {symbol} started')
            dboptstk[symbol].insert_many(data)
            print(f'Inserting option stock data for {symbol} Ended')
    
    

In [18]:
# #db_insert_optstk('STAGINGDB112021')        
# db=mongo_client['STAGINGDB122021']
# data=dbfutstk['FUTSYMBOLS'].find()
# df=pd.DataFrame(list(data))
# df.drop("_id",axis=1,inplace=True)

# symbols=list(df['Symbols'].unique())
# symbols


In [19]:
def insert_into_db(stagingdb):
    # db_insert_eq(stagingdb)
    # db_insert_futidx(stagingdb)
    # db_insert_futstk(stagingdb)
    # db_insert_optidx(stagingdb)
    db_insert_optstk(stagingdb)

In [20]:
insert_into_db('STAGINGDB122021')

Inserting option stock data for ACC started
Inserting option stock data for ACC Ended
Inserting option stock data for ADANIENT started
Inserting option stock data for ADANIENT Ended
Inserting option stock data for ADANIPORTS started
Inserting option stock data for ADANIPORTS Ended
Inserting option stock data for AMARAJABAT started
Inserting option stock data for AMARAJABAT Ended
Inserting option stock data for AMBUJACEM started
Inserting option stock data for AMBUJACEM Ended
Inserting option stock data for APOLLOHOSP started
Inserting option stock data for APOLLOHOSP Ended
Inserting option stock data for APOLLOTYRE started
Inserting option stock data for APOLLOTYRE Ended
Inserting option stock data for ASHOKLEY started
Inserting option stock data for ASHOKLEY Ended
Inserting option stock data for ASIANPAINT started
Inserting option stock data for ASIANPAINT Ended
Inserting option stock data for AUROPHARMA started
Inserting option stock data for AUROPHARMA Ended
Inserting option stock d